# Checklist Feedback

This notebook contains the evaluation prompt used to assess the checklist responses and provide feedback.

## Import

In [ ]:
%pip install -qU openai

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import threading
import pickle
import shutil
import json
import time  
import re

from openai import AzureOpenAI

KEY1 = "XXX"
KEY2 = "XXX"
API_VERSION = "XXX"
LOCATION = "XXX"
ENDPOINT = "XXX"
DEPLOYMENT = "XXX"

client = AzureOpenAI(
  azure_endpoint=ENDPOINT,
  api_key=KEY2,
  api_version=API_VERSION
)

## Prompt for evaluation

In [ ]:
prompt_template = """You are provided with a "Paper" to be submitted to the NeurIPS conference. You are assisting the authors in preparing their "Answer" to one checklist "Question". Please examine carefully the proposed author's "Answer" and the proposed author's "Justification" provided, and identify any discrepancies with the actual "Paper" content, for this specific "Question", taking into account the "Guidelines" provided to authors.

Afterwards, provide itemized, actionable feedback, based on the "Guidelines", aiming to improve the paper quality. Concentrate on a few of the most significant improvements that can be made, and write in terse technical English. While Authors' Proposed Answer is generally preferred to be a "Yes", it is acceptable to answer "No" or "NA" provided a proper Authors' Proposed Justification is given (e.g., "error bars are not reported because it would be too computationally expensive" or "we were unable to find the license for the dataset we used"). If the Authors' Proposed Answer is Yes, the Authors' Proposed Justification for the Answer should point to the section(s) within which related material for the question can be found. Note that the Authors' Proposed Justification is not expected to contain anything else (although it is fine if it contains more details).

Finally, after performing all previous steps, conclude your review with a score for this specific "Question", in a separate line (1: Everything OK or mild issues; 0.5: Needs improvements. Use this score sparingly; 0: Critical issues). Make sure that score is shown in a new line in this format "Score: score_value" and there is no content after the score.

Question:
<START OF QUESTION>
{question}
<END OF QUESTION>

Answer:
<START OF ANSWER>
{answer}
<END OF ANSWER>

Justification:
{justification}

Guidelines:
<START OF GUIDELINES>
{guideline}
<END OF GUIDELINES>

Paper:
<START OF PAPER>
{paper}
<END OF PAPER>"""

## Process evaluation for all papers

In [ ]:
def process_submission_chunk(chunk):
    for _, row in chunk.iterrows():
        sub_id = row['submission_id'] 
        df_qa = pd.read_csv(f'submissions/{sub_id}/paper_checklist.csv')
        with open(f'submissions/{sub_id}/article_dict.pickle', 'rb') as f:
            dict_paper = pickle.load(f)

        str_sections = f"Abstract:\n{dict_paper['abstract']}\n\n"
        for sec in dict_paper['sections']:
            if sec['heading'] in ['Claims', 'Limitations']:
                break
            str_sections += f"Section {sec['heading']}:\n{sec['text']}\n\n"

        # run the evaluation 3 times
        for run in range(3):  
            score_reproduce = []
            
            # run the evaluation for 15 questions
            for id_q in range(15):
                msg = prompt_template.format(
                    question=df_qa['Question'][id_q],
                    answer=df_qa['Answer'][id_q],
                    justification=df_qa['Justification'][id_q],
                    guideline=df_qa['Guidelines'][id_q],
                    paper=str_sections,
                )

                response = client.chat.completions.create(
                    model=DEPLOYMENT,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": msg},
                    ]
                )
                try:
                    output = response.choices[0].message.content
                    # Extract the score (0, 0.5, 1)
                    matches = re.findall(r'\b(0\.5|1|0)\b', output[-450:])
                    float_matches = [float(match) for match in matches][0]

                    score_reproduce.append(float_matches)
                except:
                    score_reproduce.append(0)

            df_qa[f'run-{run}'] = score_reproduce

        df_qa.to_csv(f'submissions/{sub_id}/paper_checklist.csv', index=False)
        print(sub_id, " saved!")
            
def split_dataframe(df, num_chunks):
    chunk_size = len(df) // num_chunks
    chunks = [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    return chunks

In [ ]:
df_sub_all = pd.read_csv('XXX')
num_threads = 2 # num of threads for faster computation

chunks = split_dataframe(df_sub_all, num_threads)

threads = []
for i in range(num_threads):
    thread = threading.Thread(target=process_submission_chunk, args=(chunks[i],))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()